In [1]:
import pyspark
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf, split, col, regexp_extract,regexp_replace
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml import Pipeline
from pyspark.ml.feature import  NGram, Tokenizer,CountVectorizer, StopWordsRemover
import pandas as pd
from pandas import DataFrame, Series

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
survey_dataset1 = spark.read.csv("/home/gifty/Documents/gsg_test/question2_files/question2_survey_A.csv", header=True, inferSchema=True)
survey_dataset2 = spark.read.csv("/home/gifty/Documents/gsg_test/question2_files/question2_survey_B.csv", header=True, inferSchema=True)
survey_dataset3 = spark.read.csv("/home/gifty/Documents/gsg_test/question2_files/question2_survey_C.csv", header=True, inferSchema=True)

In [4]:
survey_dataset1.show()

+---+---+---+---+----+----+----+---+---+---+---+---+---+---+---+---+---+----+----+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+----+----+---+-----+-----+----+----+----+-----+----+-----+-----+
| S1| S2| P1|P2T| P2D| P2I| P2R| Q1| Q2| Q3| Q4| Q5| Q6| Q7| Q8| Q9|V1X| SW2| SW3|QVSUM|Q10|Q11A|Q11B|Q12A|Q12B|Q13A|Q13B|Q14A|Q14B|Q15A|Q15B|Q16A|Q16B|Q17A|Q18A|Q19B|Q20B|Q21|Q22|Q23|Q24| V2| V3|Q25X|Q25Y|Q26X|Q26Y| Q27| Q28| Q29| V4|Q30_1|Q30_2|D105|DAGE|D102|DRACE|D100|PARTY|QAREA|
+---+---+---+---+----+----+----+---+---+---+---+---+---+---+---+---+---+----+----+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+----+----+---+-----+-----+----+----+----+-----+----+-----+-----+
|  1|  1|  4|  2|null|   2|null|  5|  6|  2|  2|  6|  2|  6|  5|  2|  3|   3|null|    5|  5|null|null|   6|null|   6|null|   5|null|   6|null|

In [5]:
survey_dataset2.show()

+---+---+---+----+----+----+----+----+----+----+----+---+---+---+---+----+----+---+----+----+-----+----+---+----+----+---+----+----+---+---+---+---+----+----+----+----+---+----+----+---+---+---+---+----+----+----+-----+----+-----+----+
| S1| S2| P1|P2AT|P2AD|P2AI|P2AR|P2BT|P2BD|P2BI|P2BR| Q1| Q2| Q3| Q4| Q5A| Q5B| V1|  Q6|  Q7|QVSUM|Q8_1| Q8|  Q9| Q10|Q11| Q12| Q13|Q14|Q15|Q16|Q17|Q18A|Q18B|Q19A|Q19B|Q20|Q21A|Q21B|Q22|Q23|Q24| V2|DAGE|D102|D105|DRACE|D100|PARTY|D400|
+---+---+---+----+----+----+----+----+----+----+----+---+---+---+---+----+----+---+----+----+-----+----+---+----+----+---+----+----+---+---+---+---+----+----+----+----+---+----+----+---+---+---+---+----+----+----+-----+----+-----+----+
|  1|  2|  7|null|null|null|null|   2|null|null|   2|  3|  2|  1|  1|null|   1|  3|   2|null|    6|   1|  3|null|   1|  3|null|   1|  2|  6|  1|  7|null|   4|null|   3|  7|null|   4|  4|  1|  3|  3|  11|   2|   5|    1|   1|    R|   4|
|  1|  1|  5|null|null|null|null|   1|null|null|   1|  3

In [6]:
survey_dataset3.show()

+---+---+---+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+-----+---+---+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+---+---+---+---+----+----+----+----+---+----+----+---+---+----+----+----+-----+----+----+----+-----+----+
| S1| S2| P1|P2AT|P2AD|P2AI|P2AR|P2BT|P2BD|P2BI|P2BR| Q1| Q2| Q3| Q4| Q5| V1|  Q6|  Q7|QVSUM| Q8|Q10|Q11|Q12_1|Q12_2|Q12_3|Q12_4|Q12_5|Q12_6|Q12_7|Q12_8|Q12_9|Q12_10|Q13_1|Q13_2|Q13_3|Q13_4|Q13_5|Q13_6|Q13_7|Q13_8|Q13_9|Q13_10|Q14|Q15_1|Q15_2|Q15_3|Q15_4|Q15_5|Q15_6|Q15_7|Q15_8|Q15_9|Q15_10|Q16_1|Q16_2|Q16_3|Q16_4|Q16_5|Q16_6|Q16_7|Q16_8|Q16_9|Q16_10|Q17|Q18|Q19|Q20| A21| B21| A22| B22|Q23| A24| B24|Q25|Q26|DAGE|D102|D105|DRACE|D300|D301|D100|PARTY|D400|
+---+---+---+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+

In [7]:
survey_dataset1 = survey_dataset1.toPandas()

In [8]:
survey_dataset2 = survey_dataset2.toPandas()

In [9]:
survey_dataset3 = survey_dataset3.toPandas()

In [11]:
combined = pd.concat([survey_dataset1, survey_dataset2], sort=False, ignore_index=True)

In [12]:
combined

,S1,S2,P1,P2T,P2D,P2I,P2R,Q1,Q2,Q3,...,Q14,Q15,Q16,Q17,Q18B,Q19A,Q20,Q21A,Q21B,D400
0,1,1,4,2.0,NaN,2.0,NaN,5,6,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,1.0,1.0,NaN,NaN,1,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1,1.0,1.0,NaN,NaN,1,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,6,1.0,NaN,NaN,1.0,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,2,1.0,1.0,NaN,NaN,1,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1,1,7,NaN,NaN,NaN,NaN,3,4,1,...,2.0,4.0,1.0,1.0,1.0,NaN,4.0,NaN,4.0,3.0
1596,1,1,1,NaN,NaN,NaN,NaN,5,2,4,...,1.0,1.0,4.0,4.0,2.0,NaN,2.0,NaN,6.0,2.0
1597,1,1,1,NaN,NaN,NaN,NaN,1,1,4,...,1.0,1.0,4.0,4.0,NaN,1.0,1.0,2.0,NaN,1.0
1598,1,1,7,NaN,NaN,NaN,NaN,4,7,5,...,2.0,7.0,7.0,7.0,7.0,NaN,7.0,NaN,7.0,2.0


In [13]:
final_combined = pd.concat([combined, survey_dataset3], sort=False, ignore_index=True)

In [14]:
final_combined

,S1,S2,P1,P2T,P2D,P2I,P2R,Q1,Q2,Q3,...,A21,B21,A22,B22,A24,B24,Q25,Q26,D300,D301
0,1,1,4,2.0,NaN,2.0,NaN,5,6,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,1.0,1.0,NaN,NaN,1,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1,1.0,1.0,NaN,NaN,1,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,6,1.0,NaN,NaN,1.0,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,2,1.0,1.0,NaN,NaN,1,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,1,1,6,NaN,NaN,NaN,NaN,3,5,2,...,NaN,4.0,NaN,4.0,NaN,4.0,4.0,1.0,2.0,2.0
2396,1,2,2,NaN,NaN,NaN,NaN,1,2,3,...,NaN,7.0,NaN,2.0,NaN,7.0,2.0,2.0,2.0,2.0
2397,1,1,4,NaN,NaN,NaN,NaN,4,3,2,...,NaN,4.0,NaN,5.0,NaN,4.0,4.0,1.0,2.0,2.0
2398,1,1,7,NaN,NaN,NaN,NaN,3,4,1,...,4.0,NaN,4.0,NaN,4.0,NaN,4.0,1.0,3.0,NaN


In [16]:
final_combined = spark.createDataFrame(final_combined)

In [17]:
final_combined.show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+---+---+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+----+----+---+---+---+---+-----+-----+----+----+----+-----+----+-----+-----+----+----+----+----+----+----+----+----+---+---+---+----+---+---+---+---+---+---+---+----+----+---+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+----+----+
| S1| S2| P1|P2T|P2D|P2I|P2R| Q1| Q2| Q3| Q4| Q5| Q6| Q7| Q8| Q9| V1X|SW2|SW3|QVSUM|Q10|Q11A|Q11B|Q12A|Q12B|Q13A|Q13B|Q14A|Q14B|Q15A|Q15B|Q16A|Q16B|Q17A|Q18A|Q19B|Q20B|Q21|Q22|Q23|Q24| V2| V3|Q25X|Q25Y|Q26X|Q26Y|Q27|Q28|Q29| V4|Q30_1|Q30_2|D105|DAGE|D102|DRACE|D100|PARTY|QAREA|P2AT|P2AD|P2AI|P2AR|P2BT|P2BD|P2BI|

In [18]:
final_combined.write.csv('question2_combined.csv', header=True)